# Car Sales Advertisement Analysis Dashboard

- This analysis explores a dataset of car sales advertisements to uncover trends and insights. 

- Its aim is to preprocess the data, address any of its missing values, and identify key pattterns in it.

In [138]:
# Importing neccesary packages, dataframes, and dataset for the analysis

import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np
from scipy.stats import zscore

pio.renderers.default = "vscode"

df = pd.read_csv("../vehicles_us.csv")

In [162]:
# Performing initial assessment of data

df.info()
df.describe()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 50382 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         50382 non-null  int64  
 1   model_year    50382 non-null  float64
 2   model         50382 non-null  object 
 3   condition     50382 non-null  object 
 4   cylinders     50382 non-null  float64
 5   fuel          50382 non-null  object 
 6   odometer      50382 non-null  float64
 7   transmission  50382 non-null  object 
 8   type          50382 non-null  object 
 9   paint_color   41308 non-null  object 
 10  is_4wd        24903 non-null  float64
 11  date_posted   50382 non-null  object 
 12  days_listed   50382 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.4+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,2011.0,ford f-150,good,6.0,gas,113000.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,113000.0,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [140]:
# Checking columns in the dataset

df.columns

Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed'],
      dtype='object')

# Data Preprocessing

- **Model Year** fills in missing values by grouping the data by model and using the medial year for each group: 

- **Cylinders** fill in any missing cylinder values by grouping by model and using the median.

- **Odometer** fills in any missing odometer readings by grouping by model year or a combination of year and model, then using the median or mean of each group.

In [141]:
# Checking required columns

required_columns = ['model', 'model_year', 'cylinders', 'odometer']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"Missing columns: {missing_columns}")

In [142]:
# Checking for empty groups before applying transformation

grouped = df.groupby(['model_year', 'model'])['odometer']

In [143]:
# Filling NaNs with a default value, as needed

df['odometer'].fillna(df['odometer'].median(), inplace=True)

In [144]:
# Filling missing odometer values

df['odometer'] = grouped.transform(lambda x: x.fillna(x.median() if not x.empty else np.nan))

In [145]:
# Filling any leftover missing values with the overall median

overall_median = df['odometer'].median()

df['odometer'] = df['odometer'].fillna(overall_median)

In [146]:
# Verifying that there is no NaN left

print(df['odometer'].isnull().sum())  # Should be 0

0


In [147]:
# Filling missing `model_year` values using median

df['model_year'] = df.groupby('model')['model_year'].transform(lambda x: x.fillna(x.median()))

In [148]:
# Filling missing cylinders values

df['cylinders'] = df.groupby('model')['cylinders'].transform(lambda x: x.fillna(x.median()))

In [149]:
# Filling missing odometer values

df['odometer'] = df.groupby(['model_year', 'model'])['odometer'].transform(lambda x: x.fillna(x.median()))

In [150]:
# Checking initial data distribution

print(df[['price', 'model_year']].describe())

               price    model_year
count   51525.000000  51525.000000
mean    12132.464920   2009.793954
std     10040.803015      6.099296
min         1.000000   1908.000000
25%      5000.000000   2007.000000
50%      9000.000000   2011.000000
75%     16839.000000   2014.000000
max    375000.000000   2019.000000


In [151]:
# Removing outliers by filtering the DataFrame based on Z-scores 

df_filtered = df[(np.abs(zscore(df['price'])) < 3) & (np.abs(zscore(df['model_year'])) < 3)]

In [152]:
# Verifying results

print(df.describe())

               price    model_year     cylinders       odometer   is_4wd  \
count   51525.000000  51525.000000  51525.000000   51525.000000  25572.0   
mean    12132.464920   2009.793954      6.121067  114909.428219      1.0   
std     10040.803015      6.099296      1.657457   57875.966729      0.0   
min         1.000000   1908.000000      3.000000       0.000000      1.0   
25%      5000.000000   2007.000000      4.000000   83330.000000      1.0   
50%      9000.000000   2011.000000      6.000000  113000.000000      1.0   
75%     16839.000000   2014.000000      8.000000  142077.000000      1.0   
max    375000.000000   2019.000000     12.000000  990000.000000      1.0   

       days_listed  
count  51525.00000  
mean      39.55476  
std       28.20427  
min        0.00000  
25%       19.00000  
50%       33.00000  
75%       53.00000  
max      271.00000  


In [153]:
# Adjusting thresholds for the filtered data if is empty or too small

if df_filtered.empty:
    print("Filtered data is empty. Consider adjusting Z-score thresholds.")

# Data Cleaning 

- Removing outliers in the model year and price enhances the clarity of the scatterplots.

In [154]:
# Removing outliers based on Z-scores

df = df[(np.abs(zscore(df['price'])) < 3) & (np.abs(zscore(df['model_year'])) < 3)]

In [155]:
fig = px.histogram(df, x="price")
fig.show()

In [156]:
# Re-checking columns in the dataset

df.columns

Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed'],
      dtype='object')

In [157]:
fig = px.scatter(df, x="price", y="model_year")
fig.show()

# Outliers Identification

- Statistical methods such as IQR (Interquartile Range) is used to identify outliers in the `model_year` and `price` columns.

In [158]:
# Calculating IQR for 'model_year' and 'price'

Q1_model_year = df['model_year'].quantile(0.25)

Q3_model_year = df['model_year'].quantile(0.75)

IQR_model_year = Q3_model_year - Q1_model_year

Q1_price = df['price'].quantile(0.25)

Q3_price = df['price'].quantile(0.75)

IQR_price = Q3_price - Q1_price


In [159]:
# Define outlier boundaries

lower_bound_model_year = Q1_model_year - 1.5 * IQR_model_year

upper_bound_model_year = Q3_model_year + 1.5 * IQR_model_year

lower_bound_price = Q1_price - 1.5 * IQR_price

upper_bound_price = Q3_price + 1.5 * IQR_price

# Outliers Removal

- Filtering out outliers reduces their effects on the analysis.

In [160]:
# Filter out the outliers

filtered_df = df[(df['model_year'] >= lower_bound_model_year) & 
                 (df['model_year'] <= upper_bound_model_year) & 
                 (df['price'] >= lower_bound_price) & 
                 (df['price'] <= upper_bound_price)]

# Scatterplots Update

- Updating scatterplots recreates to their latest version with changes applied.

In [161]:
# Scatter plot for model year vs price

fig = px.scatter(filtered_df, x='model_year', y='price', title='Model Year vs Price')

fig.show()

# Conclusion

- After preprocessing, trends in car prices and model years were identified.

- The cleaned dataset reveals that newer models typically have higher prices, with specific models showing consistent pricing patterns.

- As far as handling outliers: to ensure more informative visualizations, outliers were identified and removed `model_year` and `price` columns using the IQR method. This further helped in reducing the skewness while providing a clearer view of data trends.